In [2]:
!pip install qdrant-client sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.4/390.4 kB 10.4 MB/s eta 0:00:00


# Set up hybrid collection

In [4]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
import time
from google.colab import userdata

client = QdrantClient(url=userdata.get("CLUSTER_ENDPOINT"), api_key=userdata.get("QDRANT_API"))

collection_name = "day3_hybrid_search"

# Create hybrid collection
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(size=384, distance=models.Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(on_disk=False)
        )
    }
)

True

In [6]:
# Dataset de Noticias para Qdrant
my_dataset = [
    {
        "title": "Liberia: Senator Crayton Duncan Indicts Legislature for National Poverty",
        "description": """Senator Crayton O. Duncan of Sinoe County has sparked a national debate by
        attributing 80% of Liberia's underdevelopment to the Legislative body's failure to oversee
        the Executive. While public perception blames corruption for the country's state, Duncan
        argues that the lack of accountability and the 'appeasement' of the President by elected
        officials are the primary drivers of resource exploitation and poverty. He specifically
        criticized both former President Weah and current President Boakai for maintaining
        the 'old order' of personal enrichment over public service.""",
        "category": "Politics",
        "source": "frontpageafricaonline",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Thailand Budget Bureau Warns of Debt Ceiling Risks Amid Campaign Pledges",
        "description": """Thailand's Budget Bureau has flagged concerns over political parties' campaign
        promises for the 2026 election, which total an estimated 25 trillion baht. The bureau warns
        that proposed universal welfare schemes, electricity subsidies, and infrastructure projects
        could push the public-debt-to-GDP ratio beyond the 70% legal limit by 2027. Major parties
        like Klatham, the Democrats, and the People's Party have proposed massive spending plans
        that may conflict with long-term fiscal discipline and sovereign credit ratings.""",
        "category": "Economy",
        "source": "nationthailand",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Review of 'Reforming Lessons': The Impact of Nick Gibb on British Education",
        "description": """A look into the legacy of former British schools minister Nick Gibb, as detailed
        in the new book 'Reforming Lessons'. The text explores how Gibb and Michael Gove implemented
        evidence-led reforms, including phonics-based reading and the 'warm-strict' discipline
        model, which transformed English schools after 2010. The book credits the success of the
        Free Schools movement and the move away from 'child-centered' learning towards a knowledge-based
        curriculum, despite opposition from educational orthodoxies.""",
        "category": "Education",
        "source": "spectator_au",
        "pub_date": "2026-01-29"
    },
    {
        "title": "True Crime Drama 'Quiz' Hits Netflix: The Story of the Coughing Major",
        "description": """The critically acclaimed ITV drama 'Quiz' is arriving on Netflix, dramatizing
        the 2001 'Who Wants to Be a Millionaire?' cheating scandal. Starring Michael Sheen as Chris
        Tarrant and Matthew Macfadyen as Charles Ingram, the series follows the 'Coughing Major'
        who was accused of using an accomplice in the audience to win the top prize. The show
        received a 95% Rotten Tomatoes rating and is noted for its exceptional performances
        and tension-filled narrative.""",
        "category": "Entertainment",
        "source": "expresscouk",
        "pub_date": "2026-01-29"
    },
    {
        "title": "K League Unveils New Super Cup Trophy and Champions' Sleeve Patches",
        "description": """South Korea's K League has revealed the new trophy design for the 2026
        Coupang Play K League Super Cup, marking the competition's return after a 20-year hiatus.
        The match, scheduled for February 21, will feature Jeonbuk Hyundai Motors against
        Daejeon Hana Citizen. Additionally, a new custom sleeve patch system for reigning champions
        has been introduced, incorporating club-specific elements like stadium architecture
        and historical achievements rather than a generic league logo.""",
        "category": "Sports",
        "source": "yahoo_sports",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Kerala Government Approves Multimodal Logistics Parks Near Vizhinjam Port",
        "description": """In a strategic move to boost the industrial ecosystem around the Vizhinjam
        transshipment hub, the Kerala state government has granted in-principle approval for
        logistics parks. Central PSUs like Concor, Indian Oil Corporation, and the Central Warehousing
        Corporation will develop these facilities on over 95 acres of land. These parks are expected
        to act as anchor investments, attracting international shipping and industrial players
        to the region's burgeoning maritime economy.""",
        "category": "Infrastructure",
        "source": "toi",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Tree Walk Volunteers Document Rare Species in Thiruvananthapuram",
        "description": """Environmental volunteers from the 'Tree Walk' group identified and documented
        ten rare trees in the Museum compound of Thiruvananthapuram. The effort aims to promote an
        urban screening policy for indigenous and rare species, such as the Malabar ironwood and
        the Humboldt tree. Participants discussed the importance of scientific pruning and the
        protection of urban greenery against reckless cutting in the name of safety.""",
        "category": "Environment",
        "source": "toi",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Zelensky Commends Workers for Maintaining Infrastructure Under Attack",
        "description": """Ukrainian President Volodymyr Zelensky has issued a formal thank you to
        railway workers and emergency rescuers for their resilience. Despite daily Russian strikes
        targeting railway lines and energy facilities in regions like Kharkiv, Dnipro, and Chernihiv,
        logistics and power remain operational. Zelensky highlighted the names of several emergency
        service employees who have distinguished themselves in maintaining the country's backbone
        during the winter offensive.""",
        "category": "Global Conflict",
        "source": "ukrinform",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Super Bowl LX Predictions: Seahawks vs. Patriots in Prediction Markets",
        "description": """Analysis from the Kalshi prediction market shows the Seattle Seahawks as
        the favorite (68% probability) against the New England Patriots (33%) for Super Bowl LX.
        Unlike traditional sportsbooks, prediction markets allow for peer-to-peer trading on spreads
        and totals. Currently, the point total is hovering around 45.5, indicating market uncertainty
        about whether the game will be a defensive battle or a high-scoring shootout.""",
        "category": "Sports",
        "source": "nj_com",
        "pub_date": "2026-02-08"
    },
    {
        "title": "VAR Controversy in Manchester City's 2-1 Victory Over Liverpool",
        "description": """A bizarre refereeing decision marked Manchester City's win at Anfield.
        Dominik Szoboszlai was sent off in the 103rd minute for a foul on Erling Haaland, despite
        a goal being scored in the same play. Both Pep Guardiola and Haaland expressed confusion
        over the dismissal, calling for 'common sense'. Liverpool manager Arne Slot accepted the
        red card based on the rules but criticized the inconsistency regarding a previous
        non-call on Marc Guehi.""",
        "category": "Sports",
        "source": "yahoo_sports",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Breezy Johnson Wins Gold at Milan Cortina Olympics Amid Vonn's Crash",
        "description": """American skier Breezy Johnson secured the first gold medal for the U.S.
        in the women's downhill at the Milan Cortina Winter Olympics. Her victory was bittersweet
        as it was overshadowed by a frightening crash from teammate Lindsey Vonn. The games also
        marked the return of NHL players to Olympic ice, with teams from the U.S., Canada,
        and Sweden beginning their practice sessions in Milan.""",
        "category": "Sports",
        "source": "sootoday",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Sri Lanka Tops Group B in T20 World Cup After Defeating Ireland",
        "description": """Sri Lanka opened their T20 World Cup 2026 campaign with a 20-run victory
        over Ireland in Colombo. Kamindu Mendis and Kusal Mendis provided a fiery 67-run partnership
        to reach 163. Despite a hamstring injury, Wanindu Hasaranga led the bowling attack with
        3 for 25, supported by Maheesh Theekshana. The win places Sri Lanka at the top of
        the Group B points table with a net run rate of +1.000.""",
        "category": "Sports",
        "source": "india_com",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Experts Dismiss Elon Musk's Theory on Obsolescence of Retirement Saving",
        "description": """Retirement experts have labeled Elon Musk's recent comments 'nonsense'
        after the billionaire suggested that AI and robotics would make personal saving unnecessary.
        Musk envisions a 'universal high income' world of abundance, but researchers from Boston
        College argue that this ignores the realities of health expenses, market volatility,
        y potential shifts in Social Security. They emphasize that for the average worker,
        saving remains the only reliable protection against economic instability.""",
        "category": "Finance",
        "source": "benzinga",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Michigan Dominates Ohio State in Rivalry Win; Aday Mara Shines",
        "description": """The No. 2 Michigan Wolverines maintained their undefeated road record with
        an 82-61 victory over Ohio State. 7'3" center Aday Mara delivered a career-high 24 points,
        demonstrating rare passing vision and perimeter shooting for his size. Michigan's
        overwhelming physical presence in the paint and unselfish ball movement (20 assists)
        proved too much for the Buckeyes' defense, securing a season sweep for the Maize and Blue.""",
        "category": "Sports",
        "source": "si_wolverines",
        "pub_date": "2026-02-08"
    },
    {
        "title": "FBI Files Show No Evidence of Sex Trafficking Ring in Epstein Case",
        "description": """Internal Justice Department records released under the Epstein Files
        Transparency Act reveal that while the FBI found extensive proof of sexual abuse by
        Jeffrey Epstein, they found no evidence of a 'client list' or a sex trafficking ring
        serving powerful men. Investigations into thousands of videos and financial records
        did not implicate high-profile associates in criminal activity. The files clarify why
        federal authorities closed the case without additional charges against influential
        figures mentioned in public allegations.""",
        "category": "Legal",
        "source": "wishtv",
        "pub_date": "2026-02-08"
    }
]

# Step 2: Implement Dense and Sparse Encoding

In [9]:
# Dense embeddings
encoder = SentenceTransformer("all-MiniLM-L6-v2")

# Global vocabulary - automatically extends as new texts are processed
global_vocabulary = {}

# Simple sparse encoding (BM25-style)
def create_sparse_vector(text):
    """Create sparse vector from text using term frequency"""
    from collections import Counter
    import re

    # Simple tokenization
    words = re.findall(r"\b\w+\b", text.lower())
    word_counts = Counter(words)

    # Convert to sparse vector format, extending vocabulary as needed
    indices = []
    values = []

    for word, count in word_counts.items():
        if word not in global_vocabulary:
            # Add new word to vocabulary with next available index
            global_vocabulary[word] = len(global_vocabulary)

        indices.append(global_vocabulary[word])
        values.append(float(count))

    return models.SparseVector(indices=indices, values=values)

# Upload hybrid data
points = []
for i, item in enumerate(my_dataset):
    dense_vector = encoder.encode(item["description"]).tolist()
    sparse_vector = create_sparse_vector(item["description"])

    points.append(models.PointStruct(
        id=i,
        vector={"dense": dense_vector},
        sparse_vector={"sparse": sparse_vector},
        payload=item
    ))

client.upload_points(collection_name=collection_name, points=points)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


ValidationError: 1 validation error for PointStruct
sparse_vector
  Extra inputs are not permitted [type=extra_forbidden, input_value={'sparse': SparseVector(i...0, 1.0, 1.0, 1.0, 1.0])}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

# Step 3: Implement Hybrid Search with RRF
Instead of relying on the raw similarity scores from different models (which may be on different scales), RRF looks at the rank (position) of a result in each set. A formula,
 where
 is the rank and
 is a constant (default 2 in Qdrant), is used to calculate a new score, giving higher importance to items ranked highly by multiple methods.

In [ ]:
def hybrid_search_with_rrf(query_text, limit=10):
    """Perform hybrid search using Reciprocal Rank Fusion"""

    # Encode query for both dense and sparse
    query_dense = encoder.encode(query_text).tolist()
    query_sparse = create_sparse_vector(query_text)

    # Use Qdrant's built-in RRF
    response = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=query_dense,
                using="dense",
                limit=20
            ),
            models.Prefetch(
                query=query_sparse,
                using="sparse",
                limit=20
            )
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=limit
    )

    return response.points

# Test hybrid search
results = hybrid_search_with_rrf("your test query")
for i, point in enumerate(results, 1):
    print(f"{i}. {point.payload.get('title', 'No title')} (Score: {point.score:.3f})")

# Step 4: Compare Search Approaches

In [ ]:
def compare_search_methods(query_text):
    """Compare dense, sparse, and hybrid search results"""

    print(f"Query: '{query_text}'\n")

    # Dense-only search
    dense_results = client.query_points(
        collection_name=collection_name,
        query=encoder.encode(query_text).tolist(),
        using="dense",
        limit=5
    )

    # Sparse-only search
    sparse_results = client.query_points(
        collection_name=collection_name,
        query=create_sparse_vector(query_text),
        using="sparse",
        limit=5
    )

    # Hybrid search
    hybrid_results = hybrid_search_with_rrf(query_text, limit=5)

    print("DENSE SEARCH:")
    for i, point in enumerate(dense_results.points, 1):
        print(f"  {i}. {point.payload.get('title', 'No title')} ({point.score:.3f})")

    print("\nSPARSE SEARCH:")
    for i, point in enumerate(sparse_results.points, 1):
        print(f"  {i}. {point.payload.get('title', 'No title')} ({point.score:.3f})")

    print("\nHYBRID SEARCH (RRF):")
    for i, point in enumerate(hybrid_results, 1):
        print(f"  {i}. {point.payload.get('title', 'No title')} ({point.score:.3f})")

    print("-" * 50)

# Test with different query types
test_queries = [
    "exact keyword match query",
    "semantic concept query",
    "mixed keyword and concept query"
]

for query in test_queries:
    compare_search_methods(query)